## Goal

The goal of this code is to be able to label characters to first and second roles.


This is done as the inputs to predict Critics/Revenue are the movie inputs + 1st role inputs + 2nd role inputs

1) Prob: Trop de NaN? Elimine l'acteur principale. Beaucoup manque voir graphe Ali.

*NaN ne peut pas être utiliser mais plus de problème de secondary classé primary (assez de remaining data?)

2) Prob: Que faire si même valeur? Que faire si 0?

*Egaux si même valeur (pas fait)

*Si 0 ==> Missed (fait)

3) Prob: Partial_mentions donne des erreurs? Noms de famille?

*Utiliser NLTK et utiliser Mentions_Couns/Partial_counts pour membres de famille (pas fait)

4) Prob: Nom du character ne figure pas mais plutôt celui de l'acteur?

*Peut rentrer en compte mais arrive pas si souvent (pas fait)

5) Prob: "Nom" pas pris en compte à cause du manque de "" dans résumé

*Rajouter dans fonction (fait)

6) Prob: Pas toute la cast dans le dataframe

*Plus de problème (assez de remaining data?)

7) Prob: Young John vs. John characters

*Garder que le old ou le young pose problème (pas fait)

8) Autre: Ligne qui se répètent (Dr. Pramila)

*Peut simplement être filter out si pas déjà fait dans le cleaning

10) Prob: Première mention est le nom du film?

*Eliminer nom du film dans les listes. Titres éponymes donc pas possible de le faire!

## Useful libraries

In [9]:
import pandas as pd
import numpy as np
import ast
import missingno as msno
from geopy.geocoders import Nominatim
import geopandas as gpd
import re
import pycountry_convert as pc
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Data importation

In [10]:
import dataframes as RAW
import cleaning as CLEAN

In [11]:
RAW.summaries.head()

,Wiki ID,Summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [12]:
RAW.character_data.head()

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,Map ID,Character ID,Actor ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


In [13]:
RAW.movie_data_scraped.head()

,Wiki ID,Freebase ID,Movie name,Release date,Revenue,Runtime,Languages,Countries,Genres,IMDb rating,Wins,Nominations
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",4.9,NaN,2.0
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",6.0,NaN,NaN
2,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",5.9,NaN,NaN
3,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ...",6.9,1.0,6.0
4,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...",5.8,NaN,NaN


## Data quantities

Changer RAW.character_data.head() au bon dataframe filtré.
Besoin de filtre les NaN en character name

In [14]:
#Add summaries to corresponding characters (can be avoided by using RAW.summaries in functions)
merged = pd.merge(RAW.character_data, RAW.summaries, on='Wiki ID')

How many characters have NaN for name

In [15]:
nan_character_name_rows = merged['Character name'].isnull().sum()
nan_character_name_rows

166069

How many characters in total

In [16]:
merged.shape

(308485, 14)

How many movies with at least one character NaN

In [17]:
movies_with_nan_character_name = merged[merged['Character name'].isnull()]['Wiki ID'].nunique()
movies_with_nan_character_name

31882

How many movies in total

In [18]:
unique_movies_count = merged['Wiki ID'].nunique()
unique_movies_count

37779

## Parsing

Take a subset for easier testing. Dropping rows with NaN as Character Name (needed for parsing)

In [19]:
merged_test = merged.iloc[0:100,:]
merged_test = merged_test.dropna(subset=['Character name'])

Count mentions (the entire character name has to figure in the summary)

In [20]:
def count_mentions(summary, character_name):
    return summary.lower().count(character_name.lower())

merged_test['Mentions_Count'] = merged_test.apply(lambda row: count_mentions(row['Summary'], row['Character name']), axis=1)

Count partial mentions (part of the character name has to figure in the summary)

In [21]:
def count_partial_mentions(summary, character_name):
    count = 0
    summary_lower = summary.lower()
    character_name_parts = character_name.lower().split()  # Split the character name into parts

    # Count occurrences of each part of the character's name in the summary
    for part in character_name_parts:
        count += summary_lower.count(part)

    return count

merged_test['Partial_Mentions_Count'] = merged_test.apply(lambda row: count_partial_mentions(row['Summary'], row['Character name']), axis=1)

Keep only the two most partially mentionned and assigns First role to the most mentioned and second role to the other. (USELESS car outdated, il y a bien meilleur maitenant)

In [22]:
# Group by 'Wiki_ID' and find the top two actors with the most mentions for each movie
top_actors_per_movie = merged_test.groupby('Wiki ID').apply(lambda x: x.nlargest(2, 'Partial_Mentions_Count'))
# Assign roles to the top two actors for each movie
top_actors_per_movie['Role'] = ['First Role', 'Second Role'] * (len(top_actors_per_movie) // 2)
# Drop the groupby index to obtain a flat DataFrame
top_actors_per_movie = top_actors_per_movie.droplevel(0)

Keeps only the movie with no NaN cast members (USELESS car de toute manière pas toute la cast figure dans le dataframe original)

In [23]:
# Assuming 'merged' is your DataFrame containing 'Character name', 'Wiki ID', and other columns

# Find rows with NaN in 'Character name'
nan_character_name_rows = merged[merged['Character name'].isnull()]

# Extract unique Wiki IDs from rows with NaN in 'Character name'
nan_character_wiki_ids = nan_character_name_rows['Wiki ID'].unique()

# Filter rows from 'merged' where 'Character name' is not NaN or Wiki ID is not in nan_character_wiki_ids
filtered_merged = merged[(~merged['Character name'].isnull()) | (~merged['Wiki ID'].isin(nan_character_wiki_ids))]

# Now 'filtered_merged' contains rows without NaN in 'Character name' and without rows sharing the same Wiki ID as those with NaN in 'Character name'
filtered_merged_test = filtered_merged.iloc[0:100,:]

Apply the two counts to filtered_merged_test dataframe

In [24]:
filtered_merged_test['Mentions_Count'] = filtered_merged_test.apply(lambda row: count_mentions(row['Summary'], row['Character name']), axis=1)
filtered_merged_test['Partial_Mentions_Count'] = filtered_merged_test.apply(lambda row: count_partial_mentions(row['Summary'], row['Character name']), axis=1)

<ipython-input-24-263015d4f1d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_merged_test['Mentions_Count'] = filtered_merged_test.apply(lambda row: count_mentions(row['Summary'], row['Character name']), axis=1)
<ipython-input-24-263015d4f1d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_merged_test['Partial_Mentions_Count'] = filtered_merged_test.apply(lambda row: count_partial_mentions(row['Summary'], row['Character name']), axis=1)


In [25]:
# Assuming 'filtered_merged_test' is your DataFrame after filtering

# Group by 'Wiki ID' and find the top two actors with the most mentions for each movie
top_actors_per_movie = filtered_merged_test.groupby('Wiki ID').apply(lambda x: x.nlargest(2, 'Partial_Mentions_Count'))

# Assign roles dynamically based on the number of actors per movie
role_list = ['First Role', 'Second Role'] * (len(top_actors_per_movie) // 2)
if len(top_actors_per_movie) % 2 != 0:
    role_list.append('First Role')  # Adding an extra role for odd-length groups

# Assign roles to the top two actors for each movie
top_actors_per_movie['Role'] = role_list[:len(top_actors_per_movie)]

# Drop the groupby index to obtain a flat DataFrame
top_actors_per_movie = top_actors_per_movie.droplevel(0)

In [26]:
top_actors_per_movie.head()

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,Map ID,Character ID,Actor ID,Summary,Mentions_Count,Partial_Mentions_Count,Role
38,156558,/m/014k4y,2001-06-27,Jody,1978-12-30,M,1.797,/m/0x67,Tyrese Gibson,22.0,/m/0jtx5h,/m/03jnxf4,/m/01l1b90,A young 20-year-old named Jody lives with his...,25,25,First Role
37,156558,/m/014k4y,2001-06-27,Yvette,1970-09-11,F,1.650,/m/0x67,Taraji P. Henson,30.0,/m/0jtx5t,/m/03jnxj_,/m/0blbxk,A young 20-year-old named Jody lives with his...,22,22,Second Role
173,167857,/m/016fyc,1994-10,Dave Kujan,1952-05-15,M,1.910,/m/0xnvg,Chazz Palminteri,NaN,/m/045x9r0,/m/02xhr5m,/m/02t_v1,"On a ship in San Pedro Bay, a faceless figure ...",1,13,First Role
171,167857,/m/016fyc,1994-10,Kobayashi,1946-02-07,M,1.750,/m/02w7gg,Pete Postlethwaite,NaN,/m/0k6t7w,/m/02xhr51,/m/018swb,"On a ship in San Pedro Bay, a faceless figure ...",9,9,Second Role
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4,Set in the Tenacious second half of the 22nd c...,0,9,First Role


Add movies just for manual verification of results

In [27]:
# Merge based on the common Wiki ID or Freebase ID
merged_data = pd.merge(top_actors_per_movie, RAW.movie_data_scraped[['Wiki ID', 'Movie name']], on='Wiki ID', how='left')

In [28]:
merged_data.head()

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,Map ID,Character ID,Actor ID,Summary,Mentions_Count,Partial_Mentions_Count,Role,Movie name
0,156558,/m/014k4y,2001-06-27,Jody,1978-12-30,M,1.797,/m/0x67,Tyrese Gibson,22.0,/m/0jtx5h,/m/03jnxf4,/m/01l1b90,A young 20-year-old named Jody lives with his...,25,25,First Role,Baby Boy
1,156558,/m/014k4y,2001-06-27,Yvette,1970-09-11,F,1.650,/m/0x67,Taraji P. Henson,30.0,/m/0jtx5t,/m/03jnxj_,/m/0blbxk,A young 20-year-old named Jody lives with his...,22,22,Second Role,Baby Boy
2,167857,/m/016fyc,1994-10,Dave Kujan,1952-05-15,M,1.910,/m/0xnvg,Chazz Palminteri,NaN,/m/045x9r0,/m/02xhr5m,/m/02t_v1,"On a ship in San Pedro Bay, a faceless figure ...",1,13,First Role,NaN
3,167857,/m/016fyc,1994-10,Kobayashi,1946-02-07,M,1.750,/m/02w7gg,Pete Postlethwaite,NaN,/m/0k6t7w,/m/02xhr51,/m/018swb,"On a ship in San Pedro Bay, a faceless figure ...",9,9,Second Role,NaN
4,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4,Set in the Tenacious second half of the 22nd c...,0,9,First Role,Ghosts of Mars


Parsing with NLTK

Version one of first step: extract character names (unordered (to check))

In [29]:
# Example movie summaries
summaries = list(RAW.summaries['Summary'].iloc[:200])

# Function to extract character names from a given summary
def extract_character_names(summary):
    characters = {}
    sentences = sent_tokenize(summary)
    
    for sentence in sentences:
        words = word_tokenize(sentence)
        tagged_words = pos_tag(words)
        
        for i in range(len(tagged_words) - 1):
            if tagged_words[i][1] == 'NNP':  # Proper noun (assumed to be a character name)
                name = tagged_words[i][0]
                characters[name] = characters.get(name, 0) + 1
    
    sorted_characters = sorted(characters.items(), key=lambda x: x[1], reverse=True)
    return [char[0] for char in sorted_characters]

# Extract and organize character names from each movie summary
for idx, summary in enumerate(summaries, start=1):
    names = extract_character_names(summary)
    print(f"Movie {idx} character names:")
    print(names)
    print()


Movie 1 character names:
['Shlykov', 'Lyosha']

Movie 2 character names:
['Katniss', 'Peeta', 'Rue', 'District', 'Haymitch', 'Cato', 'Crane', 'Clove', 'Capitol', 'Hunger', 'Cornucopia', 'President', 'Snow', 'Thresh', 'Panem', 'Reaping', 'Primrose', 'Everdeen', 'Mellark', 'Abernathy', 'Career', 'Caesar', 'Flickerman', 'Games', 'Glimmer', 'Furious', 'Marvel', 'Seneca', 'Gamemaker']

Movie 3 character names:
['Induchoodan', 'Menon', 'Manapally', '[', 'Pavithran', 'Indulekha', 'Justice', 'Nambiar', 'Nair', 'Madhavan', 'Anuradha', 'Poovalli', 'Maranchery', 'Karunakara', 'DYSP', 'Sankaranarayanan', 'Indian', 'Civil', 'Service', 'Sudheeran', 'Saikumar', 'Ramakrishnan', 'Moopil', 'Bharathapuzha', 'Aishwarya', 'Mooppil', 'Kerala', 'Kanaka', 'Chandrabhanu', 'Jayakrishnan', 'Poovally', 'Raman', 'Nandagopal', 'Maarar']

Movie 4 character names:
['Kid', 'Charley', 'Moran', 'Brainy', 'Nellie', 'Thursday', 'New', 'Santa', 'York', 'Moose', 'Oxford', 'Christmas', 'Claus', 'Lemon', 'Drop', 'City', 'Flor

Movie 6 character names:
['Thomas', 'Baldwin', 'Kate', 'Stevens', 'Alex', 'Crawford', 'Jack', 'Agent', 'President', 'Secret', 'Service', 'Vaughan', 'Steven']

Movie 7 character names:
['Dahlia', 'Cecilia', 'Natasha', 'Veeck', 'Kyle', 'Murray', 'Jersey', 'City', 'Roosevelt', 'Island', 'Mr.', 'Hello', 'Kitty', 'Rimsky', 'Flash', 'Weeks']

Movie 8 character names:
['Hannah', 'Dominic', 'Miss', 'Lombardo', 'Leader', 'Sing', 'Jewish', 'Brooklyn', 'Christmas', 'Senior', 'Mickey', 'Dept', 'Education']

Movie 9 character names:
['Doe', 'John', 'Mitchell', 'Willoughby', 'Norton', 'Christmas', 'Eve', 'Connell', '[', 'City', 'Hall', 'Ann', 'Henry', 'D.B', 'Edward', 'Arnold', 'Colonel', 'Millsville', 'Hanson', 'Jesus', 'Christ', 'Try']

Movie 10 character names:
['Woody', 'Buzz', 'Buzzard']

Movie 11 character names:
['Lola', 'Bo', 'Nikki', 'Simona', 'Marco', 'Dotty', 'Victor', 'Texas', 'New', 'Mexico', 'Caucasian', 'Nina']

Movie 12 character names:
['Milan', 'Goran', 'Lubos', 'Eman', 'Miluska', 

Version two of first step: extract character names (ordered)

In [30]:
# Example movie summaries
summaries = RAW.summaries[['Wiki ID', 'Summary']].iloc[:600]

# Function to extract character names from a given summary
def extract_character_names(summary):
    characters = {}
    sentences = sent_tokenize(summary)
    
    for sentence in sentences:
        words = word_tokenize(sentence)
        tagged_words = pos_tag(words)
        
        for i in range(len(tagged_words) - 1):
            if tagged_words[i][1] == 'NNP':  # Proper noun (assumed to be a character name)
                name = tagged_words[i][0]
                characters[name] = characters.get(name, 0) + 1
    
    sorted_characters = sorted(characters.items(), key=lambda x: x[1], reverse=True)
    return [char[0] for char in sorted_characters]

# Create a dictionary of character names for each movie
sorted_character_names_per_movie = {}

for idx, row in summaries.iterrows():
    wiki_id = row['Wiki ID']
    summary = row['Summary']
    
    names = extract_character_names(summary)
    sorted_character_names_per_movie[wiki_id] = names

print(sorted_character_names_per_movie)

{23890098: ['Shlykov', 'Lyosha'], 31186339: ['Katniss', 'Peeta', 'Rue', 'District', 'Haymitch', 'Cato', 'Crane', 'Clove', 'Capitol', 'Hunger', 'Cornucopia', 'President', 'Snow', 'Thresh', 'Panem', 'Reaping', 'Primrose', 'Everdeen', 'Mellark', 'Abernathy', 'Career', 'Caesar', 'Flickerman', 'Games', 'Glimmer', 'Furious', 'Marvel', 'Seneca', 'Gamemaker'], 20663735: ['Induchoodan', 'Menon', 'Manapally', '[', 'Pavithran', 'Indulekha', 'Justice', 'Nambiar', 'Nair', 'Madhavan', 'Anuradha', 'Poovalli', 'Maranchery', 'Karunakara', 'DYSP', 'Sankaranarayanan', 'Indian', 'Civil', 'Service', 'Sudheeran', 'Saikumar', 'Ramakrishnan', 'Moopil', 'Bharathapuzha', 'Aishwarya', 'Mooppil', 'Kerala', 'Kanaka', 'Chandrabhanu', 'Jayakrishnan', 'Poovally', 'Raman', 'Nandagopal', 'Maarar'], 2231378: ['Kid', 'Charley', 'Moran', 'Brainy', 'Nellie', 'Thursday', 'New', 'Santa', 'York', 'Moose', 'Oxford', 'Christmas', 'Claus', 'Lemon', 'Drop', 'City', 'Florida', 'Eve', 'Year', 'Baxter', 'Home', 'Old', 'Dolls', 'Manh

Des fois les listes sont trop grandes peut être couper mais potentiel que certains characters soient pas nommés


Version one second step: Assign Primary and Secondary role.

Le premier character de la liste est First Role tous les autres figurant dans le liste Secondary Role. Pour ceux ne figurant pas dans la liste: Missed.

In [31]:
def determine_role(row):
    character_name = row['Character name']
    wiki_id = row['Wiki ID']

    # Check if character_name is a string or NaN
    if isinstance(character_name, str):
        # Get the sorted character names for the current movie using Wiki ID
        characters_for_movie = sorted_character_names_per_movie.get(wiki_id, [])

        # Check if any part of the character name matches any part of the first name in the list
        for name in characters_for_movie:
            if isinstance(name, str) and any(part in name for part in character_name.split()):
                # If the character is the first in the list
                if characters_for_movie.index(name) == 0:
                    return 'First Role'
                else:
                    return 'Secondary Role'

    return 'Missed'

# Apply the determine_role function to create the new 'Role_bis' column
RAW.character_data['Role_bis'] = RAW.character_data.apply(determine_role, axis=1)


Version two second step: Assign Primary and Secondary role.

Le premier character de la liste est First Role, ceux figurant dans le dernier quart de la liste sont second role. Le reste est missed. 

Amélioration à faire: Etablir un threshold supérieur pour les Primary role.

In [32]:
def determine_role_threshold(row):
    character_name = row['Character name']
    wiki_id = row['Wiki ID']

    # Check if character_name is a string or NaN
    if isinstance(character_name, str):
        # Get the sorted character names for the current movie using Wiki ID
        characters_for_movie = sorted_character_names_per_movie.get(wiki_id, [])

        # Set a threshold for considering characters as secondary roles
        threshold = max(2, len(characters_for_movie) // 4)  # Adjust the threshold as needed

        # Check for the first role (if list is not empty)
        if characters_for_movie and any(part in characters_for_movie[0] for part in character_name.split()):
            return 'First Role'
        else:
            # Check for secondary roles
            for name in characters_for_movie[1:]:  # Start iteration from the second item onwards
                if any(part in name for part in character_name.split()):
                    # If the character is beyond the threshold
                    if characters_for_movie.index(name) >= threshold:
                        return 'Secondary Role'

    return 'Missed'

# Apply the determine_role function to create the new 'Role_bis' column
RAW.character_data['Role_bis'] = RAW.character_data.apply(determine_role_threshold, axis=1)


Version three second step: Assign Primary and Secondary role.

Le premier character de la liste est First Role, ceux figurant dans le dernier quart de la liste sont second role. Le reste est missed. 

Amélioration à faire: Etablir un threshold supérieur pour les Primary role.

Addition par rapport à version two: characters with same family name/surname are differentiated (à vérifier que marche car pas encore testé)

In [33]:
from collections import Counter

def determine_role_threshold_nuance(row):
    character_name = row['Character name']
    wiki_id = row['Wiki ID']

    # Check if character_name is a string or NaN
    if isinstance(character_name, str):
        # Get the sorted character names for the current movie using Wiki ID
        characters_for_movie = sorted_character_names_per_movie.get(wiki_id, [])

        # Set a threshold for considering characters as secondary roles
        threshold = max(2, len(characters_for_movie) // 4)  # Adjust the threshold as needed

        # Check if there are character names for the movie and character_name
        if characters_for_movie:
            # Extract the family or shared surname from the first item in the list
            first_name = characters_for_movie[0].split()[0]  # Assumes the first item represents a family or surname

            # Create lists of names and surnames for characters in the movie
            names = [name.split()[0] for name in characters_for_movie]
            surnames = [name.split()[-1] for name in characters_for_movie]

            # Count occurrences of main character's name and surname within respective lists
            name_count = Counter(names)[character_name.split()[0]]
            surname_count = Counter(surnames)[character_name.split()[-1]]

            # Check if the main character's name or surname has the highest count within the family or surname group
            if (character_name.split()[0] == first_name and name_count >= surname_count) or \
               (character_name.split()[-1] == first_name and surname_count > name_count):
                return 'First Role'
            else:
                # Check for secondary roles based on the threshold
                for name in characters_for_movie[1:]:  # Start iteration from the second item onwards
                    if any(part in name for part in character_name.split()):
                        # If the character is beyond the threshold
                        if characters_for_movie.index(name) >= threshold:
                            return 'Secondary Role'

    return 'Missed'

# Apply the determine_role function to create the new 'Role_bis' column
RAW.character_data['Role_bis'] = RAW.character_data.apply(determine_role_threshold_nuance, axis=1)
